# Tutorial

This is a tutorial for MB-MVPA using task-fMRI data of Mixed-gamble task by Tom et al., 2007. 



### Import the MB-MVPA libarary.

Other libraries(nilean, keras, etc..) dosen't need to be imported.<br>
Because mb-mvpa has wrapping the libararies.<br>
You don't necessarily have to know fMRI libraries like nilearn and machine learning libraries like tensorflow.<br>
<b>MB-MVPA is all you need.</b>

Most of mb-mvpa are wrapping nilearn, tensorflow, Keras and etc., so warning can occur from that libraries.<br>
This page does not print warning because most of them are can be ignored.<br>
You don't need to remove the warning when you are actually using it.

In [1]:
from time import perf_counter

In [2]:
from mbmvpa.preprocessing.preprocess import DataPreprocessor

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/nilearn/glm/__init__.py:56: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  'It may change in any future release of Nilearn.', FutureWarning)


TODO: add original data download link

Data download from AWS S3, ~ <b>1GB</b> (would be under the "Mixed-gamble_task/example_data/").<br>

We provide a small subset (2 subjects) of original Tom's dataset (16 subjects). The fMRI images in the example is preprocessed by conventional fMRI preprocessing pipeline by using 
[*fmriprep*](https://fmriprep.org/en/stable/) v.20.1.0. Please refer to the [original](https://openneuro.org/datasets/ds000005/versions/00001) for more information.

In [3]:
#root = load_example_data("tom")
root = "/data2/project_modelbasedMVPA/ds000005"

### Preprocessing fMRI images and behavioral data

MB-MVPA requires primariliy preprocessed task-fMRI experiments data fromatted in conventional [BIDS format](https://bids-specification.readthedocs.io/en/stable/) 

It expects the following organized files. All the naming conventions used here conform with outputs from *fmriprep* v.20.1.0. by Poldrack lab.

The fMRI images are usually located here<br>
<i>{BIDS_ROOT}/derivatives/fmriprep/subject/session/run/func/*nii.gz</i><br>
And the behavior data are located here<br>
<i>{BIDS_ROOT}/subject/session/run/func/*.tsv</i>

In [4]:
s = perf_counter()

dm_model = 'ra_prospect'

def example_adjust(row):
    ## rename data in a row to the name which can match hbayesdm.ra_prospect requirements ##
    row["gamble"] = 1 if row["respcat"] == 1 else 0
    row["cert"] = 0
    return row

def example_filter(row):
    # include all trial data
    return True

def example_latent(row, param_dict):
    ## calculate subjectives utility for choosing Gamble over Safe option
    ## prospect theory with loss aversion and risk aversion is adopted
    modulation = (row["gain"] ** param_dict["rho"]) - (param_dict["lambda"] * (row["loss"] ** param_dict["rho"]))
    row["modulation"] = modulation
    return row


preprocessor = DataPreprocessor(bids_layout=root,
                               adjust_function=example_adjust,
                               filter_function=example_filter,
                               latent_function=example_latent,
                               dm_model=dm_model)
print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

elapsed time: 0.54 minutes


In [5]:
preprocessor.summary()

[  fMRIPrep  ] BIDS Layout: .../ds000005/derivatives/fmriprep | Subjects: 16 | Sessions: 0 | Runs: 48
[  MB-MVPA   ] BIDS Layout: ...PA/ds000005/derivatives/mbmvpa | Subjects: 16 | Sessions: 0 | Runs: 48


In [6]:
s = perf_counter()

preprocessor.preprocess(overwrite=False)

print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

elapsed time: 0.63 minutes


In [7]:
preprocessor.summary()

[  fMRIPrep  ] BIDS Layout: .../ds000005/derivatives/fmriprep | Subjects: 16 | Sessions: 0 | Runs: 48
[  MB-MVPA   ] BIDS Layout: ...PA/ds000005/derivatives/mbmvpa | Subjects: 16 | Sessions: 0 | Runs: 48


### Load data and shape check

In [8]:
from mbmvpa.data.loader import BIDSDataLoader

In [9]:
root = "/data2/project_modelbasedMVPA/ds000005"

In [10]:
s = perf_counter()

loader = BIDSDataLoader(layout=root)
X,y = loader.get_total_data(reconstruct=True)

print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

elapsed time: 0.22 minutes


In [11]:
print("X", X.shape)
print("y", y.shape)

X (11520, 46, 55, 46)
y (11520,)


In [12]:
voxel_mask = loader.get_voxel_mask()

### Fitting MVPA models & Results

In [13]:
from mbmvpa.models.mvpa_cnn import CNN

In [16]:
MVPA_model = CNN(X=X,
                y=y,
                n_patience=5,
                n_repeat=15,
                voxel_mask = voxel_mask)



In [17]:
s = perf_counter()

coeffs = MVPA_model.run()

print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

START!
Train for 125 steps, validate for 31 steps
Epoch 1/200
125/125 [==============================] - 57s 454ms/step - loss: 0.1225 - val_loss: 0.1222
Epoch 2/200
125/125 [==============================] - 35s 279ms/step - loss: 0.1208 - val_loss: 0.1226
Epoch 3/200
125/125 [==============================] - 20s 161ms/step - loss: 0.1199 - val_loss: 0.1217
Epoch 4/200
125/125 [==============================] - 19s 155ms/step - loss: 0.1181 - val_loss: 0.1218
Epoch 5/200
125/125 [==============================] - 20s 157ms/step - loss: 0.1141 - val_loss: 0.1238
Epoch 6/200
125/125 [==============================] - 20s 163ms/step - loss: 0.1097 - val_loss: 0.1263
Epoch 7/200
125/125 [==============================] - 18s 146ms/step - loss: 0.1027 - val_loss: 0.1323
Epoch 8/200
125/125 [==============================] - 19s 148ms/step - loss: 0.0952 - val_loss: 0.1429
[001] - val_loss: 0.1217


ValueError: Error when checking input: expected conv2d_4_input to have 4 dimensions, but got array with shape (46, 46)

In [ ]:
s = perf_counter()

sham_errors = MVPA_model.sham()

print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

In [ ]:
img = MVPA_model.image(save_path='.', task_name='example')